In [4]:
import pandas as pd
import json
import pyarrow.parquet as pq
import os
import numpy as np

In [5]:
path_cpv_extracted = "../data/cpv8_predictions.json"
path_meta = "../data/Metadata"
path_data_extracted = "/export/data_ml4ds/NextProcurement/Junio_2025/pliegosPlace_withExtracted"

## Data with PDF and objective extracted

In [6]:
all_dfs = []

for type in ["Insiders", "Outsiders"]:
    print(f"Processing type: {type}")
    path_data_extracted_type = f"{path_data_extracted}/{type}"

    table = pq.read_table(path_data_extracted_type, coerce_int96_timestamp_unit=None)
    
    df = table.to_pandas()
    df["origin"]  = type
    
    all_dfs.append(df)
    
df = pd.concat(all_dfs, ignore_index=True)

df.head(2)

Processing type: Insiders
Processing type: Outsiders


,place_id,ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode,title,url,id,resultado_tecnico,path_tecnico,resultado_administrativo,path_administrativo,texto_tecnico,texto_administrativo,extracted_objective,generated_objective,origin
0,https://contrataciondelestado.es/sindicacion/l...,[48821000.0],Suministro de dos servidores virtualizados par...,{'administrativo': 'https://contrataciondelest...,1,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \n \n \n \n \n \n \n \n \n \n \...,Contratación y Patrimonio\n ...,adquisición de dos servidores para maquinas vi...,El objetivo del contrato es la adquisición de ...,Insiders
1,https://contrataciondelestado.es/sindicacion/l...,[79995000.0],Servicio técnico de apoyo a la migración del c...,{'administrativo': 'https://contrataciondelest...,2,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \nMINISTERIO \nDE CULTURA \nY DEP...,\n \n \n \nMINISTERIO \nDE CULTURA \nY DEP...,la prestación de un servicio técnico de apoyo ...,El objetivo del contrato es proporcionar servi...,Insiders


## Predicted CPV

In [7]:
with open(path_cpv_extracted, "r") as f:
    cpv_data = json.load(f)
    
df_cpv = pd.DataFrame(cpv_data)

# sort by id
df_cpv = df_cpv.sort_values(by="id").reset_index(drop=True)

df_cpv.head(2)

,id,title,generated_objective,cpv_predicted
0,0,Contrato derivado del AM de Suministro de ener...,El objetivo es establecer las funciones y resp...,45310000
1,1,Suministro de dos servidores virtualizados par...,El objetivo del contrato es la adquisición de ...,30200000


## Additional metadata

In [8]:
all_meta = []

for type in ["Insiders", "Outsiders"]:
    print(f"Processing metadata for type: {type}")
    path_meta_type = f"{path_meta}/{type}"
    
    # there are several json files
    
    meta_files = [f for f in os.listdir(path_meta_type) if f.endswith(".json")]
    
    # read files and merge for this type
    type_meta = []
    for mf in meta_files:
        with open(f"{path_meta_type}/{mf}", "r") as f:
            meta = json.load(f)
            type_meta.extend(meta)
    df_meta = pd.DataFrame(type_meta)
    df_meta["origin"] = type
    df_meta["id"] = df_meta["id"].astype(int)
    
    all_meta.append(df_meta)

df_meta = pd.concat(all_meta, ignore_index=True)

# sort by id
df_meta = df_meta.sort_values(by="id").reset_index(drop=True)

df_meta.head(5)

Processing metadata for type: Insiders
Processing metadata for type: Outsiders


,place_id,url,id,criterios_adjudicacion,criterios_solvencia,condiciones_especiales,origin
0,https://contrataciondelestado.es/sindicacion/P...,{'administrativo': 'https://www.contratacion.e...,0,Criterios de adjudicación 6.3.\n6.2.2.- Criter...,"Podrán contratar con el sector público, las pe...",Ejecución del contrato 10.10.\nIndemnización d...,Outsiders
1,https://contrataciondelestado.es/sindicacion/l...,{'administrativo': 'https://contrataciondelest...,0,,,,Insiders
2,https://contrataciondelestado.es/sindicacion/l...,{'administrativo': 'https://contrataciondelest...,1,Contratación y Patrimonio BSF/jmlc Expte: 11-2...,LICITACIÓN Y ADJUDICACIÓN Cláusula 11.- Capaci...,Cláusula 8.- Plazo de ejecución y prórroga El ...,Insiders
3,https://contrataciondelestado.es/sindicacion/P...,{'administrativo': 'https://contractaciopublic...,1,,,,Outsiders
4,https://contrataciondelestado.es/sindicacion/P...,{'administrativo': 'https://www.contratacion.e...,2,Criterios de adjudicación 6.3.\n6.2.2.- Criter...,"Podrán contratar con el sector público, las pe...",Revisión de precios 10.8.Modificación del cont...,Outsiders


## Lemmas

In [9]:
df_lemmas = pd.read_parquet("../data/preprocessed/lics_for_topicmodeling_preproc.parquet")

df_lemmas.iloc[0].place_id

'https://contrataciondelestado.es/sindicacion/licitacionesPerfilContratante/13351527'

## Merge

In [10]:
# add cpv data to df by merging on generated_objective only
df_merged = df.merge(df_cpv[["generated_objective", "cpv_predicted"]], on="generated_objective", how="left")

# add additional metadata by merging on id and origin
df_merged = df_merged.merge(df_meta[["id","origin","criterios_adjudicacion", "criterios_solvencia", "condiciones_especiales"]], on=["id", "origin"], how="left")

# add lemmas 
df_merged = df_merged.merge(df_lemmas[["place_id", "lemmas"]], on="place_id", how="left")

# Check merge results
print(f"Original df shape: {df.shape}")
print(f"After CPV merge: {df_merged.shape}")
print(f"Columns after merge: {df_merged.columns.tolist()}")

df_merged.head(2)

Original df shape: (73577, 14)
After CPV merge: (293905, 19)
Columns after merge: ['place_id', 'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode', 'title', 'url', 'id', 'resultado_tecnico', 'path_tecnico', 'resultado_administrativo', 'path_administrativo', 'texto_tecnico', 'texto_administrativo', 'extracted_objective', 'generated_objective', 'origin', 'cpv_predicted', 'criterios_adjudicacion', 'criterios_solvencia', 'condiciones_especiales', 'lemmas']


,place_id,ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode,title,url,id,resultado_tecnico,path_tecnico,resultado_administrativo,path_administrativo,texto_tecnico,texto_administrativo,extracted_objective,generated_objective,origin,cpv_predicted,criterios_adjudicacion,criterios_solvencia,condiciones_especiales,lemmas
0,https://contrataciondelestado.es/sindicacion/l...,[48821000.0],Suministro de dos servidores virtualizados par...,{'administrativo': 'https://contrataciondelest...,1,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \n \n \n \n \n \n \n \n \n \n \...,Contratación y Patrimonio\n ...,adquisición de dos servidores para maquinas vi...,El objetivo del contrato es la adquisición de ...,Insiders,30200000,Contratación y Patrimonio BSF/jmlc Expte: 11-2...,LICITACIÓN Y ADJUDICACIÓN Cláusula 11.- Capaci...,Cláusula 8.- Plazo de ejecución y prórroga El ...,NaN
1,https://contrataciondelestado.es/sindicacion/l...,[79995000.0],Servicio técnico de apoyo a la migración del c...,{'administrativo': 'https://contrataciondelest...,2,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \nMINISTERIO \nDE CULTURA \nY DEP...,\n \n \n \nMINISTERIO \nDE CULTURA \nY DEP...,la prestación de un servicio técnico de apoyo ...,El objetivo del contrato es proporcionar servi...,Insiders,72000000,Tramitación del expediente Ordinaria X Urgente...,El pago del precio se realizará de manera parc...,Duración y prórroga del contrato: ► Duración E...,el contrato proporcionar servicios programació...


In [11]:
# Create a date column with year 2024 for all rows
df_merged["date"] = "2024-01-01"

# convert to datetime64[ns]
df_merged["date"] = pd.to_datetime(df_merged["date"])

# Check the result
print(f"Date column created: {df_merged['date'].dtype}")
print(f"Sample dates: {df_merged['date'].head()}")

Date column created: datetime64[ns]
Sample dates: 0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-01-01
4   2024-01-01
Name: date, dtype: datetime64[ns]


## Extract 2 CPV

In [12]:
def extract_cpv(cpv):
    if isinstance(cpv, (list, np.ndarray)):  # If it's a list or numpy array
        cpv_clean = str(cpv[0]).strip("[]") if len(cpv) > 0 else np.nan
    elif isinstance(cpv, str):  # If it's already a string
        cpv_clean = cpv.strip("[]")
    elif pd.isna(cpv):  # If it's NaN, return NaN
        return np.nan
    else:  # If it's an unexpected type (e.g., a number), return NaN
        return np.nan

    # Now process the CPV codes safely
    cpv_list = cpv_clean.split(", ")
    cpv_list = [el.split(".")[0] for el in cpv_list]  # Remove the dot and everything after it
    # if any of the cpv codes is less than 8 characters, add as many zeros as needed at the beginning until it reaches 8 characters
    for i in range(len(cpv_list)):
        if len(cpv_list[i]) < 8:
            while len(cpv_list[i]) < 8:
                cpv_list[i] = "0" + cpv_list[i]
    return list(set([el[:2] for el in cpv_list])) if cpv_list else np.nan

def extract_complete_cpv(cpv):
    if isinstance(cpv, (list, np.ndarray)):  # If it's a list or numpy array
        cpv_clean = str(cpv[0]).strip("[]") if len(cpv) > 0 else np.nan
    elif isinstance(cpv, str):  # If it's already a string
        cpv_clean = cpv.strip("[]")
    elif pd.isna(cpv):  # If it's NaN, return NaN
        return np.nan
    else:  # If it's an unexpected type (e.g., a number), return NaN
        return np.nan

    # Now process the CPV codes safely
    cpv_list = cpv_clean.split(", ")
    cpv_list = [el.split(".")[0] for el in cpv_list]  # Remove the dot and everything after it
    # if any of the cpv codes is less than 8 characters, add as many zeros as needed at the beginning until it reaches 8 characters
    for i in range(len(cpv_list)):
        if len(cpv_list[i]) < 8:
            while len(cpv_list[i]) < 8:
                cpv_list[i] = "0" + cpv_list[i]
    return list(set([el for el in cpv_list])) if cpv_list else np.nan


# rename 'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode' to 'cpv'
df_merged = df_merged.rename(columns={"ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv"})

df_merged["two_cpv"] = df_merged["cpv"].apply(extract_cpv)
df_merged["two_cpv"].value_counts()
df_merged["cpv"] = df_merged["cpv"].apply(extract_complete_cpv)

In [13]:
# remove columns path_tecnico, path_administrativo
df_final = df_merged.drop(columns=["path_tecnico", "path_administrativo"])

In [14]:
df_final.to_parquet("../data/combined_data_with_cpv_metadata_lemmas.parquet", index=False)

In [15]:
df_final["cpv"].head(5).to_json()

'{"0":["48821000"],"1":["79995000"],"2":["50118110","60100000"],"3":["72600000"],"4":["30190000"]}'

In [20]:
df_final.id

0             1
1             2
2             3
3             5
4             6
          ...  
293900    18995
293901    18996
293902    18997
293903    18998
293904    18999
Name: id, Length: 293905, dtype: int64

In [21]:
df_final[df_final.place_id.str.contains("_24")]

,place_id,cpv,title,url,id,resultado_tecnico,resultado_administrativo,texto_tecnico,texto_administrativo,extracted_objective,generated_objective,origin,cpv_predicted,criterios_adjudicacion,criterios_solvencia,condiciones_especiales,lemmas,date,two_cpv
